In [247]:
# import and preview of the dataset
import pandas as pd
data=pd.read_csv('Historical Product Demand.csv') 
df=pd.DataFrame(data)
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


<b>Obtaining information about the structure and characteristics of the df DataFrame (Historical Product Demand)</b>

In [248]:
df.shape

(1048575, 5)

In [249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   Product_Code      1048575 non-null  object
 1   Warehouse         1048575 non-null  object
 2   Product_Category  1048575 non-null  object
 3   Date              1037336 non-null  object
 4   Order_Demand      1048575 non-null  object
dtypes: object(5)
memory usage: 40.0+ MB


<i>Null values detected in Date column. To remove.<br>
Data Types for Date and Order_Demand stated as "object". To change to DateTime and Int64 accordingly.</i>

In [250]:
# updating datatypes
df['Date'] = pd.to_datetime(df['Date'])
# adding new column to see which values were converted to NaN if any
df['Order_Demand_Updated'] = pd.to_numeric(df['Order_Demand'],errors='coerce',downcast='integer')

In [251]:
df.info() # re-checking data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   Product_Code          1048575 non-null  object        
 1   Warehouse             1048575 non-null  object        
 2   Product_Category      1048575 non-null  object        
 3   Date                  1037336 non-null  datetime64[ns]
 4   Order_Demand          1048575 non-null  object        
 5   Order_Demand_Updated  1038106 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 48.0+ MB


In [252]:
data_nan = df[df['Order_Demand_Updated'].isna()]
data_nan.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Order_Demand_Updated
112290,Product_2169,Whse_A,Category_024,2012-08-09,(1),NaN
112307,Product_2132,Whse_A,Category_009,2012-11-01,(24),NaN
112308,Product_2144,Whse_A,Category_009,2012-11-01,(24),NaN
112356,Product_2118,Whse_A,Category_009,2012-03-07,(50),NaN
112357,Product_2120,Whse_A,Category_009,2012-03-07,(100),NaN


<u>Note:</u> Negative values are enclosed within brackets, rather than using a minus sign. This caused issues during the conversion from object to numeric data type, and it requires correction.

In [253]:
# Custom function to convert values with brackets to integers
def convert_to_integer(value):
    try:
        return int(value)
    except ValueError:
        # Handle values with brackets for negative numbers
        try:
            return -int(value.strip('()'))
        except ValueError:
            return None  # Handle other non-convertible values as needed

In [254]:
# adding another column to compare with primary data
df['Order_Demand_Updated2'] = df['Order_Demand'].apply(convert_to_integer)


In [255]:
df.describe(include='all')

C:\Users\gareg\AppData\Local\Temp\ipykernel_14700\2884002236.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all')


,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Order_Demand_Updated,Order_Demand_Updated2
count,1048575,1048575,1048575,1037336,1048575,1.038106e+06,1.048575e+06
unique,2160,4,33,1729,3828,NaN,NaN
top,Product_1359,Whse_J,Category_019,2013-09-27 00:00:00,1000,NaN,NaN
freq,16936,764447,481099,2075,112682,NaN,NaN
first,NaN,NaN,NaN,2011-01-08 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,2017-01-09 00:00:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,4.936488e+03,4.867427e+03
std,NaN,NaN,NaN,NaN,NaN,2.902462e+04,2.893346e+04
min,NaN,NaN,NaN,NaN,NaN,0.000000e+00,-9.990000e+05
25%,NaN,NaN,NaN,NaN,NaN,2.000000e+01,1.800000e+01



<i><b>Summary:</b> <br>The dataset consist of approximately 5 years historical data and contains 2,160 unique products distributed across 4 warehouses and belonging to 33 product categories.<br> The 'min' value in 'Order_Demand' columns indicates negative demand, likely corresponding to product return operations. It is necessary to remove both negative and corresponding positive demand entries to get real demand pattern.</i>

In [256]:
# check what NaN is for Date column
data_nan_date=df[df['Date'].isna()]
data_nan_date.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Order_Demand_Updated,Order_Demand_Updated2
45460,Product_1461,Whse_A,Category_019,NaT,10000,10000.0,10000
456373,Product_1636,Whse_A,Category_019,NaT,100,100.0,100
456374,Product_1461,Whse_A,Category_019,NaT,300,300.0,300
456375,Product_1464,Whse_A,Category_019,NaT,300,300.0,300
456376,Product_1388,Whse_A,Category_019,NaT,200,200.0,200


In [257]:
# remove rows with NaN dates
df.dropna(subset=['Date'], inplace=True)

# remove Order_Demand_Updated and Order_Demand_Updated2 as they were added to see the difference and result of our updates
df.drop(['Order_Demand_Updated','Order_Demand_Updated2'],axis=1,inplace=True)



In [258]:
# converting Order_Demand to integer
df['Order_Demand'] = df['Order_Demand'].apply(convert_to_integer)

In [259]:
# remove rows with 0 order demand
df = df[df['Order_Demand'] != 0]

In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1008664 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Product_Code      1008664 non-null  object        
 1   Warehouse         1008664 non-null  object        
 2   Product_Category  1008664 non-null  object        
 3   Date              1008664 non-null  datetime64[ns]
 4   Order_Demand      1008664 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 46.2+ MB


In [261]:
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012-07-27,100
1,Product_0979,Whse_J,Category_028,2012-01-19,500
2,Product_0979,Whse_J,Category_028,2012-02-03,500
3,Product_0979,Whse_J,Category_028,2012-02-09,500
4,Product_0979,Whse_J,Category_028,2012-03-02,500


In [262]:
# creating series (series_negative) with extracted indices with negative values
df_sorted=df['Order_Demand'].sort_values()
df_sorted.head()
negative_value_count = df[df['Order_Demand'] < 0]['Order_Demand'].count()
print("Negative Value Count:", negative_value_count)
series_negative=df_sorted[0:negative_value_count]
print(series_negative.tail())

Negative Value Count: 5899
990924   -1
603818   -1
187255   -1
768838   -1
398251   -1
Name: Order_Demand, dtype: int64


In [263]:
# Dropping rows with negative values in Order_Demand which don't have positive pairs (can explain their existance)
series_negative_pairs=[]
number_of_single_negatives=0
for index_negative, value_negative in series_negative.items():
    # Check if the absolute of negative value exists in the 'Order_Demand' column
    value_exists = (df['Order_Demand'] == abs(value_negative)).any()
    if not value_exists:
        df.drop(index=index_negative, inplace=True)
        number_of_single_negatives+=1
    else :
        series_negative_pairs.append(( index_negative,value_negative))
        
print(f"{number_of_single_negatives} rows with negative Order_Demand value were dropped as they didn't have any positive pair")


26 rows with negative Order_Demand value were dropped as they didn't have any positive pair


In [264]:
# preparing to remove negative and possitive pair. "For loop" can't be used as it's very time consuming for this quantites.

# convert all negative values in Order_Demand to positive ones (info about them beeing them negative is stored in series_negative_pairs)

df['Order_Demand'] = df['Order_Demand'].abs()

# Sort by 'Order_Demand' and then by 'Date'
df.sort_values(by=['Order_Demand', 'Date'], inplace=True)

In [265]:
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
44795,Product_0965,Whse_A,Category_006,2011-11-18,1
93055,Product_0504,Whse_J,Category_015,2011-12-05,1
130643,Product_2165,Whse_C,Category_024,2011-12-06,1
37485,Product_1699,Whse_J,Category_026,2011-12-07,1
77277,Product_1680,Whse_S,Category_021,2011-12-09,1


In [266]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1008638 entries, 44795 to 20892
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Product_Code      1008638 non-null  object        
 1   Warehouse         1008638 non-null  object        
 2   Product_Category  1008638 non-null  object        
 3   Date              1008638 non-null  datetime64[ns]
 4   Order_Demand      1008638 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 46.2+ MB


In [267]:
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
44795,Product_0965,Whse_A,Category_006,2011-11-18,1
93055,Product_0504,Whse_J,Category_015,2011-12-05,1
130643,Product_2165,Whse_C,Category_024,2011-12-06,1
37485,Product_1699,Whse_J,Category_026,2011-12-07,1
77277,Product_1680,Whse_S,Category_021,2011-12-09,1


In [268]:
# insert new column (New_ID) for incremental numbers will need in order to remove negative-positive pairs. 
df.insert(0,"New_ID",range(0,len(df)))

In [269]:
df

,New_ID,Product_Code,Warehouse,Product_Category,Date,Order_Demand
44795,0,Product_0965,Whse_A,Category_006,2011-11-18,1
93055,1,Product_0504,Whse_J,Category_015,2011-12-05,1
130643,2,Product_2165,Whse_C,Category_024,2011-12-06,1
37485,3,Product_1699,Whse_J,Category_026,2011-12-07,1
77277,4,Product_1680,Whse_S,Category_021,2011-12-09,1
...,...,...,...,...,...,...
258396,1008633,Product_1248,Whse_J,Category_019,2013-11-11,2560000
72638,1008634,Product_1341,Whse_C,Category_019,2012-02-06,3000000
20477,1008635,Product_1248,Whse_J,Category_019,2012-07-05,3000000
222266,1008636,Product_1432,Whse_S,Category_019,2013-06-14,3000000


In [270]:
# Create an empty list to store New_IDs corresponding to negative values and their possitive pairs as they will be in "New_ID-1" row
new_ids_for_negatives = []

# Iterate through series_negative_pairs
for index_negative, _ in series_negative_pairs:
    # Find the corresponding New_ID using loc
    new_id = df.loc[df.index == index_negative, 'New_ID'].values[0]
    new_ids_for_negatives.append(new_id)
    new_ids_for_negatives.append(new_id-1)

print(f"Total rows to be removed is {len(new_ids_for_negatives)}")

Total rows to be removed is 11746


In [271]:
# now removing rows where New_IDs are in new_ids_for_negatives

# Create a boolean mask to filter rows based on New_ID
mask = df['New_ID'].isin(new_ids_for_negatives)

# Use the mask to filter the DataFrame and keep rows where New_ID is not in the list
df = df[~mask]


In [272]:
# now we can drop New_ID column as it was created to remove negative-positive pairs
df=df.drop('New_ID',axis=1)

In [273]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997700 entries, 44795 to 20892
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Product_Code      997700 non-null  object        
 1   Warehouse         997700 non-null  object        
 2   Product_Category  997700 non-null  object        
 3   Date              997700 non-null  datetime64[ns]
 4   Order_Demand      997700 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 45.7+ MB


In [274]:
df.shape

(997700, 5)

In [275]:
# Saving cleanned dataframe to a new CSV file
df.to_csv('Historical_Product_Demand_Cleaned.csv',index=False)